In [ ]:
!pip install openpyxl
import pandas as pd

path_in = "./../resources/data_ignored/report.merged-sanne-binge_1705-out-congnitions-disfunctional.xlsx"

path_out = "./../resources/data_ignored/report.merged-sanne-binge_1705-out-congnitions-disfunctional-out.xlsx"
path_out_sum = "./../resources/data_ignored/report.merged-sanne-binge_1705-out-congnitions-disfunctional-sum.xlsx"
path_out_mean = "./../resources/data_ignored/report.merged-sanne-binge_1705-out-congnitions-disfunctional-mean.xlsx"
path_out_size = "./../resources/data_ignored/report.merged-sanne-binge_1705-out-congnitions-disfunctional-size.xlsx"

df = pd.read_excel(path_in)
data_top = df.head()  
    
# display  
print(data_top)  
print(len(df.index))

In [ ]:
df = pd.read_excel(path_in)

#helper column
df['sum'] = 1

In [ ]:
# create cognition columns
cogn_counts = df['COGN_FINAL'].value_counts()
print(cogn_counts)
cogn_counts.plot.bar(rot=90)

cogn_index = cogn_counts.index.tolist()

for cogn in cogn_index:
    df[cogn] = 0

for cogn in cogn_index:
    df.loc[df['COGN_FINAL'] == cogn, cogn] = 1
    
# df.to_excel(path_out)

In [ ]:
df_grouped = df.groupby('subject_id_x', as_index=False, sort=False)

df_grouped.sum().to_excel(path_out_sum)
df_grouped.mean().to_excel(path_out_mean)
df_grouped.size().to_excel(path_out_size)


In [ ]:
 df['COGN_FINAL'].count()

In [ ]:
def post_intake_graphs(df_graphs):
    post_intake = pd.DataFrame(columns=['POST_INTAKE_CAT','MEAN'])
    for x in range(1,25):
        col = 'POST_INTAKE_POR_' + str(x)
        count = df_graphs[col].notnull().sum()
        post_intake = post_intake.append([
            {'POST_INTAKE_CAT': col,
             'COUNT': count,
             'MEAN': df_graphs[col].mean(),
            }])

#     post_intake = post_intake.sort_values(by=['COUNT'], ascending=False)
    print(post_intake)
    post_intake.plot.bar(x='POST_INTAKE_CAT', y='MEAN', rot=90)
    post_intake.plot.bar(x='POST_INTAKE_CAT', y='COUNT', rot=90)
    
df_binges = df.loc[(df['objective_binge'] == True)]
print('total binges', len(df_binges.index))
post_intake_graphs(df_binges)

In [ ]:
df_non_binges = df.loc[(df['objective_binge'] != True)]
print('total non-object binges', len(df_non_binges.index))
post_intake_graphs(df_non_binges)

In [ ]:
df['date_normal'] = pd.to_datetime(df['global_date_x']).dt.date
df = df.sort_values('date_normal')

# could have been done with groupby first
def get_day(subject_id, date_normal):
    subject_rows = df.loc[df['subject_id_x'] == subject_id]
    subject_rows.sort_values('date_normal')
    ref_date = subject_rows.iloc[0]['date_normal']
    day = (date_normal - ref_date).days + 1
    return day

df['day_number'] = df.apply(lambda x: get_day(x['subject_id_x'], x['date_normal']), axis=1)

summed_date = df.groupby(['subject_id_x','day_number'], as_index=False)['sum'].sum()


In [ ]:
summed_date.to_excel("./../resources/data_ignored/report.merged-sanne-binge_1705-out-congnitions-date.xlsx")


In [ ]:
plt = summed_date.boxplot(column='sum', by='day_number')
plt.figure.savefig("./../resources/data_ignored/compliance.png")
